# Streaming de tweets en tiempo real

### Paso 1:  Autenticación 

En primer lugar dirígete a https://developer.twitter.com/en/apps y haz click en los **detalles de la aplicación** que has creado previamente. 

Vamos a crear el trozo de código que nos permitirá autenticarnos en el API de Twiter. Para facilitar la tarea, vamos a utilizar un *wrapper* llamado **Tweepy**.  
Ahí te aparecerán 4 claves que necesitaremos que incluyas en el siguiente código respetando las comillas simples:

In [7]:
import tweepy
import json
import pandas as pd


###incluye aqui tus claves unicas de Twitter
consumer_key = ''
consumer_secret= ''
access_token= '' 
access_token_secret= ''

#autenticacion del usuario
def get_auth():
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    return auth


### Paso 2:  Escucha activa de tweets y procesamiento

En este ejercicio, en vez de realizar una consulta como tal y recibir *n* tweets, vamos a quedarnos "escuchando" qué es lo que está sucediendo en Twitter en este momento en referencia a un contenido que nos interese. 

Para ello disponemos de dos eventos que se dispararán cuando el API detecte tweets que cumplen nuestro filtro.

El primer evento es **on_status**

El segundo evento es **on_data**

a) ¿Cuál crees que es la función de cada uno?

b) **Añade al dataset el número de retweets y el de favoritos**

*Pista: busca por retweet y favorite en* https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets


In [22]:
class MyStreamListener(tweepy.StreamListener):

    def on_status(self, status):
        
        print(status.text.encode("ascii", errors='replace'))  # Console output compatible con UTF-8 (emojis)
        print("="*10)
        
    def on_data(self, data):
        try:
            decoded = json.loads(data)
            if decoded['lang'] == 'en':
                print(">> Se ha detectado un tweet en castellano que cumple los filtros:")
                print(decoded['text'])
                dict_ = {'user': [], 'user_location': [], 'date_created': [],  'text': [], 'retweet_count': [], 
                'favorite_count': [], 'hashtags': [], 'user_mentions': [], 'urls': []}   

                dict_['user'].append(decoded['user']['screen_name'])
                dict_['user_location'].append(decoded['user']['location'])
                dict_['date_created'].append(decoded['created_at'])
                if decoded['truncated'] == True:
                    print(">> Tweet largo")
                    dict_['text'].append(decoded['extended_tweet']['full_text'])
                else: dict_['text'].append(decoded['text'])
                dict_['retweet_count'].append(decoded['retweet_count'])
                dict_['favorite_count'].append(decoded['favorite_count'])                            
                dict_['hashtags'].append(decoded['entities']['hashtags'])
                dict_['user_mentions'].append(decoded['entities']['user_mentions'])
                dict_['urls'].append(decoded['entities']['urls'])  
                
                df = pd.DataFrame(dict_)  

                #Si queremos ordenar los tweets por algun valor
                #df.sort_values(by='retweet_count', inplace=True, ascending=False)
                
                #creamos y pasamos los datos capturados a un csv
                df.to_csv(path_or_buf = "streaming.csv", mode="a", header=False, encoding="utf-8")

            else:
                print(">> Tweet descartado por idioma diferente")
            
                
        except Exception as e:
            print("ERROR: {}".format(e))
        finally:
            return True  # Bucle que me permite seguir escuchando


### Paso 3: Método principal - conectando los módulos

Ya tenemos todos los eventos necesarios y el método de autenticación. El siguiente paso será crear los objetos que llamen a cada uno de los módulos y activen el Stream. Recuerda escoger el hashtag con el que quieres escuchar o bien la localización

In [23]:
if __name__ == '__main__':
    print("===== Stream de tweets por hashtag o location =====")
    
    # Creamos un objeto tweepy para conectarnos al wrapper
    auth = get_auth()  # Retrieve an auth object using the function 'get_auth' above
    api = tweepy.API(auth)  # Build an API object.

    # Nos conectamos al stream
    myStreamListener = MyStreamListener()
    myStream = tweepy.Stream(auth=api.auth, listener=myStreamListener)


    print(">> Escuchando tweets...")
    
    # Se puede filtrar por hashtag o keyword, como en el notebook anterior 
    myStream.filter(track=['#ElectionDay'])

    #Si tuviésemos la versión de pago, podríamos trabajar con GPS Location
    #Para conseguir una zona de coordenadas como locations=[-123.26,33.75,-115.72,38.35] 
    #o locations=[-180,-90,180,90] dirigirse a https://boundingbox.klokantech.com/


===== Stream de tweets por hashtag o location =====
>> Escuchando tweets...
>> Se ha detectado un tweet en castellano que cumple los filtros:
RT @BorisJohnson_MP: On this important day, I would like to send a message of support to my dear friend Donald Trump:
#ElectionDay
#USElect…
>> Se ha detectado un tweet en castellano que cumple los filtros:
RT @HeyTammyBruce: Thank you @IvankaTrump &amp; @realDonaldTrump for all your hard work! I think we’re all awake, we have your backs and we’re…
>> Tweet descartado por idioma diferente
>> Tweet descartado por idioma diferente
>> Se ha detectado un tweet en castellano que cumple los filtros:
RT @GovLarryHogan: Today is #ElectionDay! Plan ahead, be safe, and make your voice heard. 

Find a vote center or drop box near you: https:…
>> Se ha detectado un tweet en castellano que cumple los filtros:
RT @BlondieOfficial: "One of the penalties for refusing to participate in politics is that you end up being governed by your inferiors.” -…
>> Se ha det

KeyboardInterrupt: ignored